### 분석 목적: 유튜브 조회 수, 좋아요, 싫어요 수를 통한 오디션 프로그램의 우승자 예측

### 분석 배경
- 2021년 12월 6일날 첫 방영한 "싱어게인2" 프로그램에서 우승자를 예측하기 위함
- 유튜브 조회 수는 '화제성'을 상징하는 지표라고 생각하고, '좋아요'와 '싫어요'는 대중들의 평가를 상징하는 지표라고 생각
- 이 지표들을 분석함으로써 실제 오디션 프로그램의 우승자를 예측해보려고 함



[url](https://www.youtube.com/watch?v=D0HfiSWP6-Y) 참고해서 적어보자


In [1]:
import numpy as np
import MySQLdb
import pandas as pd
import re
import requests
import time

from datetime import datetime
from sqlalchemy import create_engine

In [ ]:
# connect mySql
conn = MySQLdb.connect(host="******", 
                       port=3306, 
                       user='root', 
                       password='******',
                       database='singAgain',
                       charset='utf8')
# create db
#cursor.execute("CREATE DATABASE singAgain")

# create table
#sql = """CREATE TABLE youtubeInfo (time DATETIME, title TINYTEXT, 
#        viewCount int, likeCount int, favoriteCount int, commentCount int)"""
#cursor.execute(sql)
#conn.commit()

cursor = conn.cursor()

In [3]:
# api id
my_id = '***********'

In [4]:
def run(list_id):
    list_url = f'https://www.googleapis.com/youtube/v3/playlistItems?part=id,snippet,contentDetails,status&playlistId={list_id}&maxResults=50&key={my_id}'
    
    # request list_url
    result = requests.get(list_url)
    result_dict = ""

    # 정상 -> json
    if result.ok:
        result_dict = result.json()
    
    # 재생목록에서 video id 받아오기
    video_id_list = []
    for i in range(len(result_dict['items'])):
        video_id = result_dict['items'][i]['snippet']['resourceId']['videoId']
        video_id_list.append(video_id)
        
    # video info 추출
    result_values = []
    for video in video_id_list:
        video_url = f'https://www.googleapis.com/youtube/v3/videos?part=id,snippet,statistics&id={video}&key={my_id}'

        result = requests.get(video_url)
        result_dict = result.json()

        # values
        try:
            time_value = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            title_value = result_dict['items'][0]['snippet']['title']
            title_value = re.sub("[^ㄱ-힣1-9 ]+", "", title_value)
            count_value_list = list(result_dict['items'][0]['statistics'].values())

            # insert sql
            sql = f"""INSERT INTO youtubeInfo VALUES
                            ("{time_value}", "{title_value}", {count_value_list[0]}, {count_value_list[1]}, {count_value_list[2]}, {count_value_list[3]})"""
            cursor.execute(sql)
            conn.commit()
        except:
            print(video)
            continue

In [5]:
def print_time():
    print(datetime.now().strftime('%m월%d일 %H:%M'))

In [ ]:
zig_cam_list_id = "PLx_gmH-Sx5dW9mMRLNH_Y0tKLNppOQIcV"
hot_clip_list_id = "PLis5WuyWOdrUvQYJh8qRe-re8Fxoe0WtG"
list_id_1 = "PL5qRfk4DowRCPbG1pNlGbgqKlgVxzb-fv"
list_id_2 = "PL5qRfk4DowRAtIVvvpG3Bp660CTsyWOmF"

schedule.every().hour.at(":00").do(run, zig_cam_list_id)
schedule.every().hour.at(":30").do(run, zig_cam_list_id)

schedule.every().hour.at(":00").do(run, hot_clip_list_id)
schedule.every().hour.at(":30").do(run, hot_clip_list_id)

schedule.every().hour.at(":00").do(run, list_id_1)
schedule.every().hour.at(":30").do(run, list_id_1)

schedule.every().hour.at(":00").do(run, list_id_2)
schedule.every().hour.at(":30").do(run, list_id_2)


while True:
    schedule.run_pending()
    time.sleep(1)